# Shared Governance at Scale

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/shared_governance_scale/shared_governance_scale.ipynb)
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/06_advanced_workflows/shared_governance_scale/shared_governance_scale.ipynb)

## Business Scenario

Your data platform is onboarding **13 new source systems** (users, events, sessions,
purchases, refunds, subscriptions, logins, pageviews, clicks, support tickets, device
events, feature flags, daily metrics) into Bronze and Silver.

Each system needs the same baseline:
- Trim whitespace from string fields
- Drop duplicate records
- Attach lineage columns (`_lakelogic_source`, `_lakelogic_processed_at`)
- Handle soft-delete markers from source systems
- Enforce a not-null check on every primary key

Without shared governance, this means **duplicating the same YAML blocks** across
26 contracts (13 bronze + 13 silver) and hoping they stay in sync.

## Business Value

| Without Shared Governance | With Shared Governance |
|---|---|
| 26 contracts each define their own trim/dedup | One base template, applied everywhere |
| Governance change = 26 manual edits | Change the template, re-apply in one command |
| No guarantee contracts stay aligned | Template application is idempotent + auditable |
| Manual fan-out to update rules | `apply_contract_template()` propagates instantly |
| Sequential processing | Registry driver runs all 26 contracts in parallel |

---

## Goals

1. Review the folder structure and input files
2. Apply the shared base template to all 13 Silver contracts
3. Run all 26 contracts (Bronze + Silver) in parallel with a shared policy pack
4. Inspect the output — row counts, quarantined records


## Setup & File Overview

Key inputs:

```
shared_governance_scale/
├── contracts/
│   ├── _registry.yaml          ← Defines all 26 contracts (13 bronze + 13 silver)
│   ├── _shared/
│   │   └── base_silver.yaml    ← Shared baseline: lineage, dedup, quality defaults
│   ├── bronze/                 ← 13 Bronze contracts (raw ingestion)
│   └── silver/                 ← 13 Silver contracts (cleansed)
├── policy_packs/
│   └── shared_standard.yaml    ← Shared trim, dedup, not-null transformations
└── data/
    └── *.csv                   ← 13 source files (users, events, sessions, ...)
```

In [ ]:
from pathlib import Path
import subprocess
import sys
import shutil

def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        if base.name == name and base.exists():
            return base
        candidate = base / name
        if candidate.exists():
            return candidate
        candidate = base / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
        candidate = base / "lakelogic" / "examples" / "06_advanced_workflows" / name
        if candidate.exists():
            return candidate
    return cwd / name

EXAMPLE_DIR  = resolve_example_dir("shared_governance_scale")
REGISTRY     = EXAMPLE_DIR / "contracts" / "_registry.yaml"
BASE_TMPL    = EXAMPLE_DIR / "contracts" / "_shared" / "base_silver.yaml"
POLICY_DIR   = EXAMPLE_DIR / "policy_packs"

# Enumerate inputs
bronze_contracts = sorted((EXAMPLE_DIR / "contracts" / "bronze").glob("*.yaml"))
silver_contracts = sorted((EXAMPLE_DIR / "contracts" / "silver").glob("*.yaml"))
data_files       = sorted((EXAMPLE_DIR / "data").glob("*.csv"))

print(f"Bronze contracts : {len(bronze_contracts)}")
print(f"Silver contracts : {len(silver_contracts)}")
print(f"Source data files: {len(data_files)}")
print()
print("Entities:", [f.stem.replace('bronze_', '').replace('silver_', '') 
                    for f in bronze_contracts])

## Step 1: Apply the Base Template to all Silver Contracts

The base template deep-merges shared defaults into each Silver contract:
- **Appends** shared `transformations`, `quality.row_rules`, `quality.dataset_rules`
- **Injects soft-delete handling** when the source has `operation`, `deleted_at`,
  or `is_deleted` columns
- **Preserves** each contract's own entity-specific rules (list-mode = append)

This can be run as a **CLI command** or via the **Python API**:

In [ ]:
# Python API — apply the shared Silver base template
from lakelogic.tools.template_apply import apply_contract_template

results = apply_contract_template(
    base_template=BASE_TMPL,
    registry=REGISTRY,
    stage="silver",
    list_merge_keys=["transformations", "quality.row_rules", "quality.dataset_rules"],
    list_mode="append",
    soft_delete=True,
)

written = [r for r in results if r.written]
print(f"Template applied to {len(results)} Silver contracts")
print(f"Updated / written  : {len(written)}")
for r in results:
    status = "✔ written" if r.written else "  unchanged"
    print(f"  {status}  {r.contract_path.name}")

### CLI Equivalent

The same operation from the terminal:

```bash
python scripts/apply_contract_template.py \
    --base-template contracts/_shared/base_silver.yaml \
    --registry contracts/_registry.yaml \
    --stage silver \
    --list-merge-keys transformations,quality.row_rules,quality.dataset_rules \
    --list-mode append \
    --soft-delete
```

## Step 2: Run All 26 Contracts in Parallel with a Shared Policy Pack

The **Policy Pack** (`shared_standard.yaml`) adds shared runtime behaviour applied
to every contract — trim whitespace, dedup, enforce primary key not-null — without
touching the contract files themselves.

The **registry driver** discovers the dependency graph, resolves Bronze → Silver
ordering, and executes all contracts concurrently with 8 workers.

In [ ]:
# Python API — PipelineDriver
from lakelogic.cli import PipelineDriver, ContractLoader

driver = PipelineDriver(
    registry=str(REGISTRY),
    layers=["bronze", "silver"],
    policy_pack="shared_standard",
    policy_pack_dir=str(POLICY_DIR),
    max_workers=8,
)

import time
t0 = time.perf_counter()
summary = driver.run()
t1 = time.perf_counter()

total    = len(summary)
success  = sum(1 for s in summary.values() if s.get("status") == "success")
failed   = total - success

print(f"Ran {total} contracts in {t1 - t0:.1f}s with 8 parallel workers")
print(f"  Succeeded : {success}")
print(f"  Failed    : {failed}")

In [ ]:
# CLI equivalent (runs the same driver as a subprocess)
def build_driver_cmd(args):
    exe = shutil.which("lakelogic-driver")
    if exe:
        return [exe] + args
    return [sys.executable, "-m", "lakelogic.cli.driver"] + args

cmd = build_driver_cmd([
    "--registry",      str(REGISTRY),
    "--layers",        "bronze,silver",
    "--policy-pack",   "shared_standard",
    "--policy-pack-dir", str(POLICY_DIR),
    "--max-workers",   "8",
])
print("CLI command:")
print(" ".join(cmd))

## Step 3: Inspect the Output

Browse the materialised Bronze and Silver output files.

In [ ]:
import polars as pl

output_root = EXAMPLE_DIR / "output"
output_files = sorted(output_root.rglob("*.parquet")) if output_root.exists() else []

print(f"Output files written: {len(output_files)}")
total_rows = 0
for f in output_files:
    layer = f.parent.parent.name   # bronze | silver
    entity = f.parent.name
    df = pl.read_parquet(f)
    total_rows += len(df)
    print(f"  [{layer:6s}] {entity:25s} → {len(df):>4} rows")

print(f"\nTotal rows materialised: {total_rows:,}")

In [ ]:
# Preview one Silver output to see lineage columns injected by the template
silver_output = output_root / "silver"
silver_files  = sorted(silver_output.rglob("*.parquet")) if silver_output.exists() else []

if silver_files:
    sample_file = silver_files[0]
    df = pl.read_parquet(sample_file)
    print(f"Sample Silver output: {sample_file.parent.name}")
    print(f"Columns: {df.columns}")
    print()
    print(df.head(5))
else:
    print("No Silver outputs found — run Step 2 first.")

---

## Summary

| Step | What happened |
|---|---|
| **Base template** | 13 Silver contracts enriched with shared lineage, dedup, soft-delete — by changing one file |
| **Policy pack** | Shared trim + PK not-null applied at runtime — zero YAML duplication |
| **Parallel driver** | 26 contracts processed concurrently — Bronze before Silver, DAG-ordered |
| **Scale** | Pattern works identically for 500+ contracts |

**The key insight**: governance rules live in **one place** (`base_silver.yaml` +
`shared_standard.yaml`). When a new compliance requirement arrives, you update
one file and re-run `apply_contract_template()` — all contracts are updated instantly.
